Here I tried to fine-tune a roberta model on the reddit dataset.

In [ ]:
! gdown 1U0pQDPYFG1m4ajKU_knjZCQmDMh9jQBp
#! gdown 1J_oVuyf-SlzO1oTRhkB2ci9GbNbZ8ZRp

Downloading...
From: https://drive.google.com/uc?id=1U0pQDPYFG1m4ajKU_knjZCQmDMh9jQBp
To: /content/train-balanced-sarcasm.csv
100% 253M/253M [00:02<00:00, 125MB/s]


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Not connected to a GPU
Your runtime has 37.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
import pandas as pd
import numpy as np

reddit_data = pd.read_csv('train-balanced-sarcasm.csv')
reddit_data['comment'] = reddit_data['comment'].dropna()
reddit_data['label'] = reddit_data['label'].dropna()
#colnames=['sentiment', 'num', 'date', 'topic','name','tweet'] 
#twitter_data = pd.read_csv('sentiment140.csv', names=colnames)
#reddit_data

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 71.6 MB/s 
     |████████████████████████████████| 101 kB 9.5 MB/s 
     |████████████████████████████████| 6.6 MB 67.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 5.2 MB/s 
     |████████████████████████████████| 212 kB 74.0 MB/s 
     |████████████████████████████████| 140 kB 73.6 MB/s 
     |████████████████████████████████| 1.1 MB 73.0 MB/s 
     |████████████████████████████████| 127 kB 81.4 MB/s 
     |████████████████████████████████| 144 kB 78.7 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
     |██████████████

In [ ]:
sarcastic_words = ["obviously", "clearly", "totally", "duh", "everyone knows", "right because", "yeah obviously", "yes because", "yeah because","shitlord",
                    " definitely ", "of course", "surely", "how dare", "duh", " gee ", "for sure", "amirite", "good thing that","how i love", "what a surprise"]

In [ ]:
import re

def clean_com(com):
  #com = com.lower().replace(',', '')
  com = re.sub(r'http\S+', '', com)
  com = re.sub("@[A-Za-z0-9_]+","", com)
  com = re.sub("#[A-Za-z0-9_]+","", com)
  return re.sub(r'[^A-Za-z\s]', '', com)

In [ ]:
from datasets.dataset_dict import DatasetDict
from sklearn.model_selection import train_test_split
from datasets import Dataset

X = []
Y = []

for com,y in zip(reddit_data['comment'].tolist(), reddit_data['label'].tolist()):
  if type(com) is str and str(y).isdigit(): #and not any(x in com for x in sarcastic_words):
    x = clean_com(com)
    if 20 < len(x) < 150:      
      X.append(x)
      Y.append(int(y))

X = X[20000:]
Y = Y[20000:]
X1 = []
Y1 = []
c0 = 0
c1 = 0

for x,y in zip(X,Y):
  if y == 0: 
    if c0 >= 20000: continue
    c0 += 1
  else: 
    if c1 >= 20000: continue
    c1 += 1
  X1.append(x)
  Y1.append(y)

In [ ]:
a = [x for x,y in zip(X1,Y1) if y == 1]
b = [x for x,y in zip(X1,Y1) if y == 0]
print(len(a),len(b))
#a[20:30]
#a[:50]



20000 20000


In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X1,Y1,train_size=0.9,test_size=0.1,random_state=42)

d = {'train':Dataset.from_dict({'label':y_train,'text':x_train}),
     'test':Dataset.from_dict({'label':y_test,'text':x_test})}

dataset = DatasetDict(d)

train_args = {
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'sliding_window': False,
    'max_seq_length': 75,
    'learning_rate': 0.00005,
    'weight_decay': 0.01,
    'warmup_ratio': 0.2,
    'max_grad_norm': 1.0,
    'num_train_epochs': 1,
    'train_batch_size': 32,
    'save_model_every_epoch': False,
    'save_steps': 4000,
    'fp16': True,
    'output_dir': '/outputs/',
    'evaluate_during_training': True,
}

In [ ]:
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import load_metric
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)
#small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(30000))
#small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(10000))

model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2)
training_args = TrainingArguments(output_dir="test_trainer",evaluation_strategy="epoch",
                                  num_train_epochs=3, warmup_ratio=0.2, weight_decay=0.01,
                                  per_device_train_batch_size=32,save_steps=4000)
metric = load_metric("accuracy")

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 1,
  "use_cach

  0%|          | 0/36 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file https://huggingface.co/roberta-base/r

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 36000
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3375


Epoch,Training Loss,Validation Loss,Accuracy
1,0.607700,0.604393,0.686750


The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 4000
  Batch size = 8


Epoch,Training Loss,Validation Loss,Accuracy
1,0.607700,0.604393,0.686750


In [ ]:
trainer.model.save_pretrained("reddit_model1")

In [ ]:
import shutil

shutil.move('/content/reddit_model1', "/content/drive/My Drive/")